<a href="https://colab.research.google.com/github/BAmbica39/CustomerChurnAmericanExpress/blob/main/American_Express_Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
 import numpy as np
 import pandas as pd
 import tensorflow as tf

In [30]:
tf.__version__

'2.18.0'

In [31]:
dataset = pd.read_excel("/content/American Express Dataset.xlsx")
X = dataset.iloc[:, 0:-1].values
y = dataset.iloc[:, -1].values

In [32]:
print(X)

[[553 'Delhi' 'Female' ... 4 1 274150]
 [447 'Bengaluru' 'Male' ... 4 1 519360]
 [501 'Delhi' 'Female' ... 4 1 545501]
 ...
 [627 'Mumbai' 'Female' ... 4 0 494067]
 [600 'Bengaluru' 'Female' ... 2 1 109375]
 [553 'Delhi' 'Male' ... 4 1 180031]]


In [33]:
from sklearn.preprocessing import LabelEncoder
lable_encoder = LabelEncoder()
X[:, 2] = lable_encoder.fit_transform(X[:, 2])

In [34]:
print(X)

[[553 'Delhi' 0 ... 4 1 274150]
 [447 'Bengaluru' 1 ... 4 1 519360]
 [501 'Delhi' 0 ... 4 1 545501]
 ...
 [627 'Mumbai' 0 ... 4 0 494067]
 [600 'Bengaluru' 0 ... 2 1 109375]
 [553 'Delhi' 1 ... 4 1 180031]]


In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [36]:
X

array([[0.0, 1.0, 0.0, ..., 4, 1, 274150],
       [1.0, 0.0, 0.0, ..., 4, 1, 519360],
       [0.0, 1.0, 0.0, ..., 4, 1, 545501],
       ...,
       [0.0, 0.0, 1.0, ..., 4, 0, 494067],
       [1.0, 0.0, 0.0, ..., 2, 1, 109375],
       [0.0, 1.0, 0.0, ..., 4, 1, 180031]], dtype=object)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [46]:
print(X_train.shape)
print(y_train.shape)

(7941, 11)
(7941,)


ANN

In [54]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

ann = Sequential()
ann.add(Input(shape=(11,)))
ann.add(Dense(units=5, activation='relu'))
ann.add(Dense(units=5, activation='relu'))
ann.add(Dense(units=1, activation='sigmoid'))  # binary output

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ann.fit(X_train, y_train, batch_size=32, epochs=120)


Epoch 1/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7991 - loss: 0.5526
Epoch 2/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7972 - loss: 0.4819
Epoch 3/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8006 - loss: 0.4481
Epoch 4/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7934 - loss: 0.4457
Epoch 5/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7925 - loss: 0.4362
Epoch 6/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7942 - loss: 0.4367
Epoch 7/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7962 - loss: 0.4304
Epoch 8/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8053 - loss: 0.4258
Epoch 9/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8102 - loss: 0.4248
Epoch 10/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8250 - loss: 0.4092
Epoch 11/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8181 - loss: 0.4205
Epoch 12/120
249/249 ━━━━━━━━━━━━━━━━━━━━

predictions

single prediction

In [56]:
print(ann.predict(sc.transform([[0.0, 1.0, 0.0, 501, 0, 32, 2, 0.0, 4, 1, 545501]])) >0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
[[False]]


In [58]:
y_pred= ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 0]
 [0 1]]


In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1525   48]
 [ 239  174]]


0.8554884189325277